In [ ]:
import pandas as pd
import re
import os
from statistics import mode
import numpy as np
path = 'data/'
from tqdm import tqdm
import hashlib
import random

# Rename files
The obtained data files from the SDSS API were saved as 'result (x).csv' by default. This code was used to automatically rename them to the respecting year and month.

In [ ]:
files = [f for f in os.listdir(path) if f.endswith('.csv') and f.startswith("result")]

for file_name in files:
    print(f'-> Checking {file_name}')
    file_path = os.path.join(path, file_name)

    file = pd.read_csv(file_path, encoding="latin-1", nrows=20)
    year = (file.iloc[19, 3])
    month = (file.iloc[19, 4])

    if not year or not month or pd.isnull(year) or pd.isnull(month) or pd.isna(year) or pd.isna(
            month) or year == "nan" or month == "nan":
        print(f"Skipped {file_name}")
        print(file)
        continue

    new_file_name = f'{int(year)}-{int(month)}.csv'
    new_file_path = os.path.join(path, new_file_name)

    try:
        os.rename(file_path, new_file_path)
        print(f'Renamed {file_name} to {new_file_name}')
    except IndexError:
        print(f'Error: Unable to extract year and month from {file_name}')

# Format Files
Multiple preprocessing steps have been undertaken, as seen here. Line breaks, redundant whitespaces, missing or wrong values have been deleted. 

In [ ]:
files = [f for f in os.listdir(path) if f.endswith('.csv')]

for file_name in files:
    print(f"Starting with {file_name}")
    file_path = os.path.join(path, file_name)
    sdss = pd.read_csv(file_path, on_bad_lines="skip", encoding="latin-1",
                       lineterminator="\n",
                       dtype={"statement": str, "runtime": float, "resultsize": 'Int64', "yy": 'Int64',
                              "mm:": 'Int64', "dd": 'Int64'})

    sdss["statement"] = sdss["statement"].replace("\n", "", regex=True).replace(r'\s+', ' ', regex=True).str.lower().str.strip()

    sdss.dropna(inplace=True)
    
    sdss = sdss[~pd.to_numeric(sdss['statement'], errors='coerce', downcast="float").notnull()]
    sdss = sdss[sdss['runtime'] > 0.0]
    sdss["resultsize"] = sdss["resultsize"].astype(int)
    sdss["dd"] = sdss["dd"].astype(int)
    sdss["mm"] = sdss["mm"].astype(int)
    sdss["yy"] = sdss["yy"].astype(int)

    minYear = int(sdss["yy"].min())
    minMonth = int(sdss["mm"].min())
    minDay = int(sdss["dd"].min())
    maxYear = int(sdss["yy"].max())
    maxMonth = int(sdss["mm"].max())
    maxDay = int(sdss["dd"].max())
    count = (sdss["statement"].count())
    print(f"{file_name} ranging from {minYear}-{minMonth}-{minDay} to {maxYear}-{maxMonth}-{maxDay} with {count} lines")

    sdss.to_csv(file_path, index=False)

# Delete Similar
All files combined resulted in over 150 million queries. Upon manual investigation, it could be seen that many queries are very similar to each other. Therefore, the following code was applied to reduce the similarities to mitigate overfitting of models. For that, from all queries that were the same without numbers or had the first 50 % or 100 characters in common, one was randomly sampled. Hasing was used to speed up the process.

In [ ]:
def compute_hash(statement):
    hash_object = hashlib.sha256(statement.encode())
    return hash_object.hexdigest()

Sample one of all queries that are the same without digits:

In [ ]:
files = [f for f in os.listdir(path) if f.endswith(".csv")]
final_df = pd.read_csv(f"{path}empty.csv")
for file_name in tqdm(files):
    file_path = os.path.join(path, file_name)
    df = pd.read_csv(file_path, on_bad_lines="skip", encoding="latin-1",
                       lineterminator="\n",
                       dtype={"statement": str, "runtime": float, "resultsize": 'Int64', "yy": 'Int64',
                              "mm:": 'Int64', "dd": 'Int64'})
    final_df = pd.concat([final_df, df], ignore_index=True, sort=False)

    similar_statements = {}
    for i,statement in enumerate(final_df["statement"]):
        no_digits = re.sub(r'[0-9]', '', statement)
        statement_hash = compute_hash(no_digits)
        if statement_hash not in similar_statements:
            similar_statements[statement_hash] = [i]
        else:
            similar_statements[statement_hash].append(i)
    unique_statements = [random.choice(statements) for statements in similar_statements.values()]
    final_df = final_df[final_df.index.isin(unique_statements)]

print(len(final_df))
final_df.to_csv(f"{path}full.csv", index=False)

Sample one of all queries that are the same in the first half:

In [ ]:
final_df = pd.read_csv(f"{path}full.csv", on_bad_lines="skip", encoding="latin-1",
                       lineterminator="\n",
                       dtype={"statement": str, "runtime": float, "resultsize": 'Int64', "yy": 'Int64',
                              "mm:": 'Int64', "dd": 'Int64'})
print(f"initial df: {len(final_df)}")
final_df.reset_index()
similar_statements = {}
for i,statement in enumerate(final_df["statement"]):
    statement_len=int(len(statement)*0.5)
    statement_hash = compute_hash(statement[:statement_len])
    if statement_hash not in similar_statements:
        similar_statements[statement_hash] = [i]
    else:
        similar_statements[statement_hash].append(i)

# Randomly sample one statement from each group
unique_statements = [random.choice(statements) for statements in similar_statements.values()]
final_df = final_df[final_df.index.isin(unique_statements)]

print(f"final df: {len(final_df)}")
final_df.to_csv(f"{path}full.csv", index=False)

Sample one of all queries that have the first 100 characters in common:

In [ ]:
final_df = pd.read_csv(f"{path}full.csv", on_bad_lines="skip", encoding="latin-1",
                       lineterminator="\n",
                       dtype={"statement": str, "runtime": float, "resultsize": 'Int64', "yy": 'Int64',
                              "mm:": 'Int64', "dd": 'Int64'})
print(f"initial df: {len(final_df)}")
final_df.reset_index()
similar_statements = {}
for i,statement in enumerate(final_df["statement"]):
    statement_hash = compute_hash(statement[:100])
    if statement_hash not in similar_statements:
        similar_statements[statement_hash] = [i]
    else:
        similar_statements[statement_hash].append(i)

# Randomly sample one statement from each group
unique_statements = [random.choice(statements) for statements in similar_statements.values()]
final_df = final_df[final_df.index.isin(unique_statements)]

print(f"final df: {len(final_df)}")
final_df.to_csv(f"{path}full.csv", index=False)

Train and test data split:

In [ ]:
final_df = final_df.sample(frac=1)

train_size = int(0.9 * len(final_df))
train_df = final_df.iloc[:train_size]
test_df = final_df.iloc[train_size:]

train_df["statement"] = train_df["statement"].str.slice(0,512)
test_df["statement"] = test_df["statement"].str.slice(0,512)

train_df.to_csv(f"{path}train.csv", index=False)
test_df.to_csv(f"{path}test.csv", index=False)

In [ ]:
print(len(train_df))
print(len(test_df))